In [1]:
import pandas as pd
import numpy as np
import importlib
import sys
import time
import tqdm
import logging
from utils import *

from importlib import reload
import utils
reload(utils)
from utils import *

pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [2]:

def hold(actions):
    # encourage selling for profit and liquidity
    next_probable_action = np.argsort(actions)[1]
    if next_probable_action == 2 and len(agent.inventory) > 0:
        max_profit = stock_prices[t] - min(agent.inventory)
        if max_profit > 0:
            sell(t)
            actions[next_probable_action] = 1 # reset this action's value to the highest
            return 'Hold', actions

def buy(t):
    if agent.balance > stock_prices[t]:
        agent.balance -= stock_prices[t]
        agent.inventory.append(stock_prices[t])
        return 'Buy: ${:.2f}'.format(stock_prices[t])

def sell(t):
    if len(agent.inventory) > 0:
        agent.balance += stock_prices[t]
        bought_price = agent.inventory.pop(0)
        profit = stock_prices[t] - bought_price
        global reward
        reward = profit
        return 'Sell: ${:.2f} | Profit: ${:.2f}'.format(stock_prices[t], profit)


In [3]:

model_name = 'DQN'
stock_name = '0050_2008_2018'
window_size = 10
num_episode = 1
initial_balance = 50000

stock_prices = stock_close_prices(stock_name)
stock_margin = stock_margins(stock_name)
trading_period = len(stock_prices) - 1  # 訓練期間，input stock data的總日期
returns_across_episodes = []
num_experience_replay = 0
action_dict = {0: 'Hold', 1: 'Buy', 2: 'Sell'}

logging.basicConfig(filename=f'logs/{model_name}_training_{stock_name}.log', filemode='w',
                    format='[%(asctime)s.%(msecs)03d %(filename)s:%(lineno)3s] %(message)s', 
                    datefmt='%m/%d/%Y %H:%M:%S', level=logging.INFO)

logging.info(f'Trading Object:           {stock_name}')
logging.info(f'Trading Period:           {trading_period} days')
logging.info(f'Window Size:              {window_size} days')
logging.info(f'Training Episode:         {num_episode}')
logging.info(f'Model Name:               {model_name}')
logging.info('Initial Portfolio Value: ${:,}'.format(initial_balance))

# select learning model
model = importlib.import_module(f'agents.{model_name}')
agent = model.Agent(state_dim=window_size + 3, balance=initial_balance)

In [9]:
stock_margin.iloc[0]

MarginPurchaseBuy                   828.000
MarginPurchaseCashRepayment           0.000
MarginPurchaseSell                  487.000
MarginPurchaseTodayBalance         9262.000
MarginPurchaseYesterdayBalance     8921.000
OffsetLoanAndShort                  141.000
ShortSaleBuy                        260.000
ShortSaleSell                       313.000
ShortSaleTodayBalance              1457.000
ShortSaleYesterdayBalance          1404.000
b_total                          115863.000
s_total                          123668.000
b_s_ratio                             1.067
buy_volume                              NaN
sell_volume                             NaN
bs_ratio                                NaN
Name: 2008-01-02, dtype: float64

In [10]:

import utils
reload(utils)
from utils import *

for e in tqdm.tqdm(range(1, num_episode + 1)):
    logging.info(f'\nEpisode: {e}/{num_episode}')

    agent.reset() # reset to initial balance and hyperparameters
    state = generate_combined_state(0, window_size, stock_prices, stock_margin, agent.balance, len(agent.inventory)) 
    display(pd.DataFrame(state))
    # 將prince_state與portfolio_state 橫向串連起來橫向串連，作為input state
    # print(state)

    for t in range(1, trading_period + 1):
        if t % 1000 == 0:
            logging.info(f'\n-------------------Period: {t}/{trading_period}-------------------')

        reward = 0
        next_state = generate_combined_state(t, window_size, stock_prices, stock_margin, agent.balance, len(agent.inventory))
        display(pd.DataFrame(next_state))
        previous_portfolio_value = len(agent.inventory) * stock_prices[t] + agent.balance
        # print(t,'\ninventory',agent.inventory,'\ninventory len',len(agent.inventory),'\nstock prices',stock_prices[t], \
        #     '\ninventory*stockprices + balance = ',len(agent.inventory)*stock_prices[t],'+', agent.balance,\
        #     '\nprevious portfolio value',previous_portfolio_value,'\n')
        
        if model_name == 'DDPG':
            actions = agent.act(state, t)
            action = np.argmax(actions)
        else:
            actions = agent.model.predict(state)[0]
            action = agent.act(state)
            
        # print('actions\n',actions)
        # print('action\n',action)
        # execute position
        logging.info('Step: {}\tHold signal: {:.4} \tBuy signal: {:.4} \tSell signal: {:.4}'.format(t, actions[0], actions[1], actions[2]))
        if action != np.argmax(actions): logging.info(f"\t\t'{action_dict[action]}' is an exploration.")
        if action == 0: # hold
            execution_result = hold(actions)
        if action == 1: # buy
            execution_result = buy(t)      
        if action == 2: # sell
            execution_result = sell(t)        
        
        # check execution result
        if execution_result is None:
            reward -= treasury_bond_daily_return_rate() * agent.balance  # missing opportunity
        else:
            if isinstance(execution_result, tuple): # if execution_result is 'Hold'
                actions = execution_result[1]
                execution_result = execution_result[0]
            logging.info(execution_result)    
                        
        # print('---',execution_result,'\n')
        # calculate reward
        current_portfolio_value = len(agent.inventory) * stock_prices[t] + agent.balance
        unrealized_profit = current_portfolio_value - agent.initial_portfolio_value
        reward += unrealized_profit

        agent.portfolio_values.append(current_portfolio_value)
        agent.return_rates.append((current_portfolio_value - previous_portfolio_value) / previous_portfolio_value)

        done = True if t == trading_period else False
        agent.remember(state, actions, reward, next_state, done)

        # update state
        state = next_state

        # experience replay
        if len(agent.memory) > agent.buffer_size:
            num_experience_replay += 1
            loss = agent.experience_replay()
            logging.info('Episode: {}\tLoss: {:.2f}\tAction: {}\tReward: {:.2f}\tBalance: {:.2f}\tNumber of Stocks: {}'.format(e, loss, action_dict[action], reward, agent.balance, len(agent.inventory)))
            agent.tensorboard.on_batch_end(num_experience_replay, {'loss': loss, 'portfolio value': current_portfolio_value})

        if done:
            portfolio_return = evaluate_portfolio_performance(agent, logging)
            returns_across_episodes.append(portfolio_return)


  0%|          | 0/1 [00:00<?, ?it/s]


 -10 0 

period
 MarginPurchaseBuy                   828.000
MarginPurchaseCashRepayment           0.000
MarginPurchaseSell                  487.000
MarginPurchaseTodayBalance         9262.000
MarginPurchaseYesterdayBalance     8921.000
OffsetLoanAndShort                  141.000
ShortSaleBuy                        260.000
ShortSaleSell                       313.000
ShortSaleTodayBalance              1457.000
ShortSaleYesterdayBalance          1404.000
b_total                          115863.000
s_total                          123668.000
b_s_ratio                             1.067
buy_volume                              NaN
sell_volume                             NaN
bs_ratio                                NaN
Name: 2008-01-02, dtype: float64


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500,4.095,10.820,-13.816



 -9 1 

period
 MarginPurchaseBuy                   696.000
MarginPurchaseCashRepayment           0.000
MarginPurchaseSell                  553.000
MarginPurchaseTodayBalance         9405.000
MarginPurchaseYesterdayBalance     9262.000
OffsetLoanAndShort                  297.000
ShortSaleBuy                        482.000
ShortSaleSell                       312.000
ShortSaleTodayBalance              1287.000
ShortSaleYesterdayBalance          1457.000
b_total                          116595.000
s_total                          124713.000
b_s_ratio                             1.070
buy_volume                              NaN
sell_volume                             NaN
bs_ratio                                NaN
Name: 2008-01-03, dtype: float64


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.246,4.076,10.820,-13.816


...

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.921,0.371,0.416,0.167,0.458,0.399,0.277,0.835,0.483,0.690,4.131,10.819,0.000



 51 61 

period
 MarginPurchaseBuy                   992.000
MarginPurchaseCashRepayment           5.000
MarginPurchaseSell                  651.000
MarginPurchaseTodayBalance        18484.000
MarginPurchaseYesterdayBalance    18148.000
OffsetLoanAndShort                  186.000
ShortSaleBuy                         62.000
ShortSaleSell                       235.000
ShortSaleTodayBalance              2981.000
ShortSaleYesterdayBalance          2808.000
b_total                          147016.000
s_total                          162519.000
b_s_ratio                             1.105
buy_volume                              NaN
sell_volume                             NaN
bs_ratio                                NaN
Name: 2008-04-08, dtype: float64


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.371,0.416,0.167,0.458,0.399,0.277,0.835,0.483,0.690,0.399,4.124,10.817,0.693


  0%|          | 0/1 [00:14<?, ?it/s]


KeyboardInterrupt: 

In [19]:

state = generate_combined_state(0, window_size, stock_prices, stock_margin, agent.balance, len(agent.inventory)) 
state

period
 [[1.65600000e+03 0.00000000e+00 9.74000000e+02 1.85240000e+04
  1.78420000e+04 2.82000000e+02 5.20000000e+02 6.26000000e+02
  2.91400000e+03 2.80800000e+03 2.31726000e+05 2.47336000e+05
  2.13470966e+00            nan            nan            nan]
 [1.65600000e+03 0.00000000e+00 9.74000000e+02 1.85240000e+04
  1.78420000e+04 2.82000000e+02 5.20000000e+02 6.26000000e+02
  2.91400000e+03 2.80800000e+03 2.31726000e+05 2.47336000e+05
  2.13470966e+00            nan            nan            nan]
 [1.65600000e+03 0.00000000e+00 9.74000000e+02 1.85240000e+04
  1.78420000e+04 2.82000000e+02 5.20000000e+02 6.26000000e+02
  2.91400000e+03 2.80800000e+03 2.31726000e+05 2.47336000e+05
  2.13470966e+00            nan            nan            nan]
 [1.65600000e+03 0.00000000e+00 9.74000000e+02 1.85240000e+04
  1.78420000e+04 2.82000000e+02 5.20000000e+02 6.26000000e+02
  2.91400000e+03 2.80800000e+03 2.31726000e+05 2.47336000e+05
  2.13470966e+00            nan            nan            n

f:\GoogleDrive\NCCU\Paper\DeepRL_Trading\utils.py:26: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


array([[  0.5       ,   0.5       ,   0.5       ,   0.5       ,
          0.5       ,   0.5       ,   0.5       ,   0.5       ,
          0.5       ,   0.5       ,   4.09451119,  10.81977828,
        -13.81551056]])